In [3]:
import pandas as pd

In [18]:
methyldata = pd.read_csv("methyl.csv",index_col=0)
exprdata = pd.read_csv("expr.csv",index_col=0)
assigndata = pd.read_csv("assign.csv",usecols=[1,2],index_col=0)
assigndata

,cluster.id
subjects,
subject1,1
subject2,2
subject3,1
subject4,1
subject5,1
...,...
subject496,1
subject497,1
subject498,1


In [19]:
exprdata

,ACACA,ACVRL1,AKT1,AKT1S1,ANXA1,AR,ARAF,ASNS,ATM,BAD,...,VHL,WWTR1,XBP1,XRCC1,XRCC5,YAP1,YBX1,YWHAB,YWHAE,YWHAZ
subject1,-0.778773,6.661350,0.601915,-1.381072,0.276366,6.826198,5.217787,-0.531270,-0.685574,4.965979,...,4.997389,5.227471,-1.173942,4.990185,4.739730,0.752404,4.925204,-0.332289,4.740385,1.588527
subject2,-1.202237,6.599935,-1.059473,-1.343091,0.567053,2.980298,-1.060058,-0.543807,4.112408,-0.377627,...,5.324622,-0.575675,-0.521891,1.270019,5.396299,1.926090,0.432328,0.453741,0.124431,1.516044
subject3,-0.351933,7.037106,-0.169862,-0.600338,0.467504,4.409149,5.523396,-0.820251,-0.896702,6.273091,...,6.548016,6.983861,1.174447,5.292834,5.405190,1.720668,5.243290,0.383033,7.387739,1.114908
subject4,-0.560752,6.302922,-1.095674,-0.335570,-1.799302,4.858635,4.789491,-0.335323,-1.509982,5.800878,...,4.780794,6.124618,-1.631147,5.213470,5.813819,0.552651,4.457551,-0.836184,5.890996,-0.108535
subject5,-0.260576,5.974995,-1.597704,-1.239989,-0.848312,6.202194,3.625550,-0.896659,-0.772299,5.427887,...,5.509940,5.235778,-1.466720,5.391635,5.309465,1.113457,5.062244,-1.151481,5.118148,-0.504209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
subject496,-0.933188,6.758515,0.141599,1.175414,-0.199922,5.404964,4.587526,-2.250117,-0.043880,4.791108,...,4.960378,4.375743,-0.822462,6.340625,5.713636,1.483030,5.739724,0.294931,5.237734,1.493332
subject497,-0.351743,6.307647,-0.395698,-0.410632,-0.335842,5.095289,5.294062,0.064668,-0.599051,4.903005,...,6.760614,6.283541,0.793421,5.089189,4.882482,0.584146,5.546557,-0.167474,5.152645,1.066775
subject498,-0.555563,6.201945,0.369510,-0.527820,1.141942,4.788988,4.594360,-1.039993,-0.339906,6.553227,...,4.372792,5.008954,-0.950863,4.932677,5.589072,1.423199,4.820329,0.054766,6.264235,0.935781
subject499,-0.496976,6.736355,0.309428,1.002468,-1.421916,0.913188,0.544914,-2.264783,4.140739,1.007924,...,4.496152,0.002271,-0.513729,1.362927,5.841151,0.990882,-0.240316,0.078770,0.380601,1.431541


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Leer datos
df = pd.read_csv('expr.csv')

# Suponiendo que la primera columna es una columna de etiquetas (por ejemplo, nombres de genes)
# y todas las demás columnas son datos numéricos. Ajusta según tu CSV.
df = df.iloc[:, 1:]

# Convertir todas las columnas a tipo float
df = df.apply(pd.to_numeric, errors='coerce')

# Lidiar con valores NaN (si los hay). Aquí simplemente los lleno con 0, pero podrías manejarlo de otra manera.
df.fillna(0, inplace=True)

data = df.values
n_samples, n_genes = data.shape

# Normalizar los datos (esto es crucial para un buen entrenamiento)
data = (data - data.mean(axis=0)) / data.std(axis=0)

# Convertir a tensores
tensor_data = torch.FloatTensor(data)

# Definir el generador y el discriminador
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()  # Tanh funciona bien para datos normalizados
        )
        
    def forward(self, x):
        return self.net(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)

# Instanciar modelos y optimizadores
gen = Generator(100, n_genes)
disc = Discriminator(n_genes)
gen_optimizer = optim.Adam(gen.parameters(), lr=0.0002)
disc_optimizer = optim.Adam(disc.parameters(), lr=0.0002)
criterion = nn.BCELoss()

# Entrenamiento
n_epochs = 10000
batch_size = 64

for epoch in range(n_epochs):
    for idx in range(0, n_samples, batch_size):
        # Datos reales y etiquetas reales
        real_data = tensor_data[idx:idx+batch_size]
        current_batch_size = real_data.size(0)
        real_labels = torch.ones(current_batch_size, 1)
        
        # Datos falsos y etiquetas falsas
        noise = torch.randn(current_batch_size, 100)
        fake_data = gen(noise)
        fake_labels = torch.zeros(current_batch_size, 1)
        
        # Entrenar discriminador
        disc_optimizer.zero_grad()
        
        real_preds = disc(real_data)
        real_loss = criterion(real_preds, real_labels)
        
        fake_preds = disc(fake_data)
        fake_loss = criterion(fake_preds, fake_labels)
        
        disc_loss = real_loss + fake_loss
        disc_loss.backward()
        disc_optimizer.step()
        
        # Entrenar generador
        gen_optimizer.zero_grad()
        
        noise = torch.randn(current_batch_size, 100)
        fake_data = gen(noise)
        fake_preds = disc(fake_data)
        
        gen_loss = criterion(fake_preds, real_labels)  # Queremos que el discriminador piense que estos datos son reales
        gen_loss.backward()
        gen_optimizer.step()

    # Imprimir estado del entrenamiento
    if (epoch+1) % 100 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} | Disc Loss: {disc_loss.item()} | Gen Loss: {gen_loss.item()}")

# Una vez entrenada, puedes generar datos sintéticos
noise = torch.randn(1, 100)
synthetic_data = gen(noise)
print(synthetic_data)


Epoch 100/10000 | Disc Loss: 0.11042317748069763 | Gen Loss: 2.607304096221924
Epoch 200/10000 | Disc Loss: 0.03652320057153702 | Gen Loss: 6.1525044441223145
Epoch 300/10000 | Disc Loss: 0.003343258984386921 | Gen Loss: 7.043646812438965
Epoch 400/10000 | Disc Loss: 0.014414506033062935 | Gen Loss: 8.356715202331543
Epoch 500/10000 | Disc Loss: 0.002773153129965067 | Gen Loss: 10.663679122924805
Epoch 600/10000 | Disc Loss: 0.004810021724551916 | Gen Loss: 9.026785850524902
Epoch 700/10000 | Disc Loss: 0.000711931730620563 | Gen Loss: 9.217474937438965
Epoch 800/10000 | Disc Loss: 0.001228748937137425 | Gen Loss: 10.089400291442871
Epoch 900/10000 | Disc Loss: 0.009620037861168385 | Gen Loss: 12.503063201904297
Epoch 1000/10000 | Disc Loss: 6.939322702237405e-06 | Gen Loss: 12.602293968200684
Epoch 1100/10000 | Disc Loss: 6.691618182230741e-06 | Gen Loss: 12.196562767028809
Epoch 1200/10000 | Disc Loss: 2.1861840195924742e-06 | Gen Loss: 13.224803924560547
Epoch 1300/10000 | Disc Loss